In [27]:
using Random
using AutoMLPipeline
using AutoMLPipeline.FeatureSelectors
using AutoMLPipeline.EnsembleMethods
using AutoMLPipeline.CrossValidators
using AutoMLPipeline.DecisionTreeLearners
using AutoMLPipeline.Pipelines
using AutoMLPipeline.BaseFilters
using AutoMLPipeline.SKPreprocessors
using AutoMLPipeline.Utils
ENV["COLUMNS"]=1000;

In [40]:
# Julia MLs
jrf = RandomForest()
jada = Adaboost()
jt = PrunedTree()
vote = VoteEnsemble()
stack = StackEnsemble()
best = BestLearner();
ohe = OneHotEncoder();

In [88]:
# Scikitlearn Learners
skrf = SKLearner("RandomForestClassifier")
sksvc = SKLearner("SVC")
sklsvc = SKLearner("LinearSVC")
skada = SKLearner("AdaBoostClassifier")
skgb = SKLearner("GradientBoostingClassifier");

In [42]:
# Scikitlearn preprocessors
pca = SKPreprocessor("PCA")
kpca = SKPreprocessor("KernelPCA")
fa = SKPreprocessor("FactorAnalysis")
fica = SKPreprocessor("FastICA")
rbs = SKPreprocessor("RobustScaler");
pt = SKPreprocessor("PowerTransformer");

In [43]:
# Column Selectors
cat = CatFeatureSelector()
num = NumFeatureSelector()
disc = CatNumDiscriminator();

In [44]:
using CSV
profbdata = CSV.read("profb.csv")
first(profbdata,5)

,Home.Away,Favorite_Points,Underdog_Points,Pointspread,Favorite_Name,Underdog_name,Year
,String,Int64,Int64,Float64,String,String,Int64
1,away,27,24,4.0,BUF,MIA,89
2,at_home,17,14,3.0,CHI,CIN,89
3,away,51,0,2.5,CLE,PIT,89
4,at_home,28,0,5.5,NO,DAL,89
5,at_home,38,7,5.5,MIN,HOU,89


In [36]:
X = profbdata[:,2:end] 
Y = profbdata[:,1] |> Vector;

In [95]:
p1 = @pipeline ((num |> rbs |> pca)+(num |> rbs |> fa)+(num |> rbs |> fica)+(cat |> ohe )) |> sklsvc
sum(fit_transform!(p1,X,Y) .== Y) /length(Y)

0.7991071428571429

In [96]:
acc(X,Y) = score(:accuracy,X,Y)
crossvalidate(p1,X,Y,acc)

fold: 1, 83.5820895522388
fold: 2, 77.61194029850746
fold: 3, 64.70588235294117
fold: 4, 77.61194029850746
fold: 5, 67.16417910447761
fold: 6, 73.13432835820896
fold: 7, 79.1044776119403
fold: 8, 73.52941176470588
fold: 9, 79.1044776119403
fold: 10, 62.68656716417911


(mean = 73.82352941176471, std = 6.926457357166602, folds = 10)

In [91]:
p2 = @pipeline (num |> pca) + num + (num|> fica) + (cat |> ohe) |> sklsvc
sum(fit_transform!(p1,X,Y) .== Y) /length(Y)

0.7991071428571429

In [94]:
crossvalidate(p2,X,Y,acc)

fold: 1, 55.223880597014926
fold: 2, 67.16417910447761
fold: 3, 36.76470588235294
fold: 4, 65.67164179104478
fold: 5, 41.7910447761194
fold: 6, 67.16417910447761
fold: 7, 82.08955223880598
fold: 8, 70.58823529411765
fold: 9, 67.16417910447761
fold: 10, 74.6268656716418


(mean = 62.82484635645303, std = 14.199400776537406, folds = 10)